# Trick Taking Game

## Introduction

A trick-taking game is a card game in which players take turns playing cards and the highest card played each round wins the trick. There is often a trump suit that beats all other suits, and players must follow suit if possible. The game is played over a series of rounds, with each round consisting of a number of tricks. The player or team that wins the most tricks wins the game. There are many different trick-taking games, but they all share some common elements. The most common trick-taking games are Bridge, Whist, Hearts, and Spades.
 
In this project, I'll be looking at simulated data from one round of a trick-taking game. The data was generated using a simple trick-taking game with the following rules:
- The game is played with a standard deck of 52 cards.
- There are four players, each of whom is dealt 8 cards.
- There is a trump suit dealt randomly at the beginning of the round.
 
The data consists of the number information about each players hands (number of high cards (J,Q,K,A), number of low cards (2,3,4,5), percent of trump cards, suit impurity, if the player was leading the round, and how many tricks they won). The goal is to predict the number of tricks each player will win in the round.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

## Data Preparation
We only want to look at the player using the stratgy SimpleWinTricks, not the random players.
We also want to normalize the high card and low card count to be a percentage of the total cards in the hand.

In [ ]:
df = pd.read_csv("data/output-4-8.csv")

In [ ]:
df = df[df["strategy"] == "SimpleWinTricks"]
df["high_card_percentage"] = df["high_card_count"] / df["num_tricks"]
df["low_card_percentage"] = df["low_card_count"] / df["num_tricks"]
df["leading"] = df["leading"].astype(int)
df.head()

In [ ]:
# Drop unnecessary columns
df = df.drop(
    [
        "strategy",
        "trump_suit",
        "num_players",
        "num_tricks",
        "high_card_count",
        "low_card_count",
    ],
    axis=1,
)

In [ ]:
df.describe()

## Model - Linear Regression
We will use a Linear Regression model to predict the number of tricks each player will win in the round.

In [ ]:
X = df.drop(["tricks_taken"], axis=1)
y = df["tricks_taken"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

## Evaluate

In [ ]:
lr.score(X_test, y_test)

In [ ]:
lr.coef_

In [ ]:
root_mean_squared_error(y_test, lr.predict(X_test))

The coefficients of the model will give us an idea of which features are most important in predicting the number of tricks won. We can also look at the mean squared error of the model to see how well it performs on the test data.

The coeffients are as follows:
- Suit Impurity: -2.87
- Trump Percentage: 5.39
- Leading: 0.165
- High Card Percentage: 3.37
- Low Card Percentage: -1.31

From this we can see that the Trump Percentage and High Card Percentage are the most important features in predicting the number of tricks won. The Suit Impurity and Low Card Percentage are less important, and Leading is the least important. The data was normalized, but not standardized, so the coefficients are not directly comparable.

Suit Impurity and Low Card Percentage are negatively correlated. This makes sense since having more low cards in your hand is generally a disadvantage, and having a hand with more of the same suit (less impurity) is an advantage.

The root mean squared error of the model is 0.97, which means that the model is off by about 1 trick on average. This is not bad considering that the number of tricks won is an integer value between 0 and 8.

In [ ]:
y_predict = lr.predict(X_test)
y_actual = y_test

# Plot the predicted vs actual
plt.scatter(y_predict, y_actual)
plt.plot([0, 6], [0, 6], "r-", lw=1)
plt.xlabel("Predicted Tricks Taken")
plt.ylabel("Actual Tricks Taken")
plt.title("Predicted vs Actual Tricks Taken")
plt.show()

## Model - Random Forest

In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
rf.score(X_test, y_test)

In [ ]:
rf.feature_importances_

In [ ]:
root_mean_squared_error(y_test, rf.predict(X_test))

In [ ]:
y_predict = rf.predict(X_test)
y_actual = y_test

# Plot the predicted vs actual
plt.scatter(y_predict, y_actual)
plt.plot([0, 6], [0, 6], "r-", lw=1)
plt.xlabel("Predicted Tricks Taken")
plt.ylabel("Actual Tricks Taken")
plt.title("Predicted vs Actual Tricks Taken")
plt.show()